# Manejo de errores

## # C2_S3 · Manejo de errores en Requests + BeautifulSoup (Demo profesor)
Meta: raise_for_status, timeout+backoff, validar Content-Type/tamaño, arreglar encoding.


## 🎙️ Guion (12–15 min)
0–3’  raise_for_status + política retry/skip/fail  
3–7’  timeout + backoff (429 y 5xx)  
7–10’ validar Content-Type y tamaño  
10–12’ encoding correcto → soup  
12–15’ mini-reto Slack


In [2]:
import requests
from bs4 import BeautifulSoup

import csv
import json

import time, random

## 1) Función para procesar un producto

#### Recordar ejemplo función que procesa un producto

In [3]:
from urllib.parse import urljoin

BASE = "http://books.toscrape.com/"

def extraer_producto(product):
    """Recibe un artículo <article.product_pod> y devuelve sus datos en un diccionario."""
    # Nombre del libro (con protección contra None)
    h3 = product.find("h3")
    a_tag = h3.find("a") if h3 else None
    title = a_tag["title"] if a_tag else "Título no encontrado"

    # Precio
    price_tag = product.find("p", class_="price_color")
    price = price_tag.get_text() if price_tag else "Precio no encontrado"

    # Imagen (url relativa → absoluta con urljoin)
    img_tag = product.find("div", class_="image_container").find("img")
    image_rel = img_tag["src"] if img_tag else ""
    image_url = urljoin(BASE, image_rel)

    return {"title": title, "price": price, "image_url": image_url}


## 2) Función para recorrer páginas (paginación)

Aquí usamos la función anterior dentro de un bucle:

El {} es un marcador de posición. Es un espacio que se llenará más tarde con el número de página que quieres visitar. 

In [5]:
import requests, time
from bs4 import BeautifulSoup

BASE_URL = "http://books.toscrape.com/catalogue/category/books_1/page-{}.html"


product_list = []

#for page in range(1, n_paginas + 1):
for page in range(47,53):   # Recordar que la paginación está solamente hasta la página 50 
    url = BASE_URL.format(page)

    response = requests.get(url, timeout=10)
    soup = BeautifulSoup(response.text, "html.parser")
    products = soup.select("article.product_pod")


    for product in products:
        datos = extraer_producto(product)
        product_list.append(datos)

    time.sleep(1)  # pausa entre páginas # No saturar el servidor y que sea dinámica la forma en que se escrapea
    print(f"✅ Página {page} procesada ({len(products)} productos).")




✅ Página 47 procesada (20 productos).
✅ Página 48 procesada (20 productos).
✅ Página 49 procesada (20 productos).
✅ Página 50 procesada (20 productos).
✅ Página 51 procesada (0 productos).
✅ Página 52 procesada (0 productos).


#### También al extraer datos de un producto, se puede hacer exepciones por si no encuentra cualquiera de los atributos en ese bucle

In [6]:
import requests, time
from bs4 import BeautifulSoup

BASE_URL = "http://books.toscrape.com/catalogue/category/books_1/page-{}.html"


product_list = []

#for page in range(1, n_paginas + 1):
for page in range(47,53):   # Recordar que la paginación está solamente hasta la página 50 
    url = BASE_URL.format(page)
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status() #<-  Si aparece une error para código s 4xx y 5xx
        soup = BeautifulSoup(response.text, "html.parser")
        products = soup.select("article.product_pod")
    except Exception as e:
        print(f"❌ Error en la página {page}: {e}")
        continue

    for product in products:
        try:
            datos = extraer_producto(product)
            product_list.append(datos)
        except Exception as ex:
            print("Error extrayendo datos de un producto:", ex)

    time.sleep(1)  # pausa entre páginas
    print(f"✅ Página {page} procesada ({len(products)} productos).")




✅ Página 47 procesada (20 productos).
✅ Página 48 procesada (20 productos).
✅ Página 49 procesada (20 productos).
✅ Página 50 procesada (20 productos).
❌ Error en la página 51: 404 Client Error: Not Found for url: http://books.toscrape.com/catalogue/category/books_1/page-51.html
❌ Error en la página 52: 404 Client Error: Not Found for url: http://books.toscrape.com/catalogue/category/books_1/page-52.html


#### Guardar datos

In [7]:
import csv

archivo = "productos_con_manejo_errores.csv"

with open(archivo, "w", newline="", encoding="utf-8") as f:
    # Definir el "esqueleto" del CSV
    writer = csv.DictWriter(f, fieldnames=["title", "price", "image_url"])
    
    # Escribir los encabezados
    writer.writeheader()
    
    # Escribir cada producto
    writer.writerows(product_list)

print(f"Scraping ético completado: {len(product_list)} productos guardados en {archivo}")



Scraping ético completado: 80 productos guardados en productos_con_manejo_errores.csv


# Stop

#### aplicación directa de la política de timeouts + reintentos con backoff.

#### 1. Timeout siempre

In [10]:
timeout  = 0.1
r = requests.get(url, timeout=timeout)

#### 2. Backoff exponencial + jitter

👉 Si falla:

Primer reintento: espera ≈ 1s + pequeño jitter.

Segundo reintento: ≈ 2s + jitter.

Tercero: ≈ 4s + jitter.

El random.uniform(0, 0.3) evita que muchos clientes reintenten al mismo tiempo (efecto estampida).

In [15]:
base_delay = 1  # segundos
attempt = 3
wait = base_delay * (2 ** (attempt - 1)) + random.uniform(0, 0.3)

wait


4.114933094595319

👉 Si el servidor dice explícitamente “espera X segundos”, tú obedeces.
Esto es crucial con errores 429 Too Many Requests.

#### 3. Respeto a Retry-After

In [17]:
ra = r.headers.get("Retry-After")
if ra:
    wait = float(ra)

print(f"Esperando {wait:.2f} segundos antes de reintentar...")

Esperando 4.11 segundos antes de reintentar...


#### 4. Funciones de apoyo



should_retry(status): define qué códigos son reintentarbles → 429 o 5xx.

is_html(resp): sirve como filtro, asegurando que recibes HTML y no una página de error o PDF (aunque en este snippet no lo llamaste, está listo por si lo quieres usar).

#### 5. Lógica final



Intenta hasta max_attempts.

Si consigue 200 OK, devuelve la respuesta.

Si es un error 5xx o 429, aplica backoff y reintenta.

Si es otro 4xx (404, 403…), no reintenta y hace raise_for_status().

Si después de todos los intentos falla → lanza la última excepción.

In [27]:
import time, random
import requests
from bs4 import BeautifulSoup

def should_retry(status):
    return (status == 429) or (500 <= status < 600)

def fetch_with_policy(url, max_attempts=3, base_delay=1.0, timeout=10):
    """GET con timeout, raise_for_status, backoff y respeto de Retry-After."""
    attempt, total_wait = 0, 0.0
    last_exc = None
    while attempt < max_attempts:
        attempt += 1
        t0 = time.time()
        try:
            r = requests.get(url, timeout=timeout)
            dt = (time.time() - t0) * 1000
            print(f"[{attempt}] {r.status_code} {dt:.0f}ms {url}")
            if r.status_code == 200:
                return r
            if should_retry(r.status_code):
                ra = r.headers.get("Retry-After")
                if ra:
                    wait = float(ra)
                else:
                    wait = base_delay * (2 ** (attempt - 1)) + random.uniform(0, 0.3)
                print(f"  ↳ retry en {wait:.1f}s")
                time.sleep(wait)
                total_wait += wait
                continue
            # 4xx u otros → no reintentar
            r.raise_for_status()
        except requests.RequestException as e:
            last_exc = e
            print(f"  ⚠️ excepción: {e}")
            wait = base_delay * (2 ** (attempt - 1)) + random.uniform(0, 0.3)
            print(f"  ↳ retry en {wait:.1f}s")
            time.sleep(wait)
            total_wait += wait
    raise last_exc or RuntimeError("Max reintentos agotados")


🫡 Este ejemplo ilustra perfecto la regla de oro:
“Lento es suave, y suave es rápido”: mejor reintentar con calma que martillar el servidor.

In [25]:
url = "https://httpbin.org/status/500"
try:
    fetch_with_policy(url, max_attempts=3)
except Exception as e:
    print("Falló después de reintentos:", e)


[1] 500 505ms https://httpbin.org/status/500
  ↳ retry en 1.1s
[2] 500 369ms https://httpbin.org/status/500
  ↳ retry en 2.0s
[3] 500 357ms https://httpbin.org/status/500
  ↳ retry en 4.2s
Falló después de reintentos: Max reintentos agotados


👉 Qué harás: verás `raise_for_status()` en acción con 404 y 500, y un 429 con `Retry-After`.


In [29]:
# 404 → corta rápido
try:
    # _ = fetch_with_policy("https://httpbin.org/status/404", timeout=5)
    r = requests.get("https://httpbin.org/status/404", timeout=5)
    r.raise_for_status()
except requests.HTTPError as e:
    print("404 atrapado:", e)

# 500 → backoff (reintentos)
try:
    _ = fetch_with_policy("https://httpbin.org/status/500", max_attempts=3, base_delay=1)
except Exception as e:
    print("Tras 3 intentos de 500:", e)

# 429 con Retry-After
try:
    _ = fetch_with_policy("https://httpbin.org/response-headers?Retry-After=3&status=429",
                          max_attempts=2, base_delay=1)
except Exception as e:
    print("429 finalizó:", e)


404 atrapado: 404 Client Error: NOT FOUND for url: https://httpbin.org/status/404
[1] 500 354ms https://httpbin.org/status/500
  ↳ retry en 1.2s
[2] 500 361ms https://httpbin.org/status/500
  ↳ retry en 2.1s
[3] 500 577ms https://httpbin.org/status/500
  ↳ retry en 4.2s
Tras 3 intentos de 500: Max reintentos agotados
[1] 200 355ms https://httpbin.org/response-headers?Retry-After=3&status=429


#### Content-Type y el tamaño antes de meter el contenido a BeautifulSoup

![s](contenType2.png)

👉 Qué harás: validarás Content-Type y tamaño antes de parsear.


In [31]:
#ct = content Type

def is_html(resp, min_len=150):
    ct = (resp.headers.get("Content-Type") or "").lower()
    return ("html" in ct) and (len(resp.content) >= min_len)

In [32]:
# HTML válido
r_html = requests.get("https://httpbin.org/html", timeout=10)
print("HTML válido?", is_html(r_html, min_len=120), r_html.headers.get("Content-Type"))

# JSON (no HTML)
r_json = requests.get("https://httpbin.org/json", timeout=10)
print("HTML válido?", is_html(r_json, min_len=120), r_json.headers.get("Content-Type"))

# Bytes pequeños (contenido diminuto)
r_bytes = requests.get("https://httpbin.org/bytes/50", timeout=10)
print("HTML válido?", is_html(r_bytes, min_len=120), r_bytes.headers.get("Content-Type"))


HTML válido? True text/html; charset=utf-8
HTML válido? False application/json
HTML válido? False application/octet-stream


In [36]:
import requests
from bs4 import BeautifulSoup

def fetch_html(url, timeout=10, min_len=200):
    """Descarga y valida que la respuesta sea HTML suficiente antes de parsear."""
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()

    # Validar Content-Type
    ct = (r.headers.get("Content-Type") or "").lower()
    if "html" not in ct:
        print(f"❌ Saltado: Content-Type no es HTML ({ct})")
        return None

    # Validar tamaño mínimo
    if len(r.content) < min_len:
        print(f"❌ Saltado: respuesta demasiado pequeña ({len(r.content)} bytes)")
        return None

    # Opcional: centinelas de bloqueo
    texto = r.text.lower()
    if "access denied" in texto or "captcha" in texto:
        print("❌ Saltado: parece un bloqueo/captcha")
        return None

    return BeautifulSoup(r.text, "html.parser")

# === Ejemplo de uso ===
url = "https://books.toscrape.com/"
soup = fetch_html(url)

if soup:
    print(f"Título de la página {url}:", soup.title.get_text(strip=True))

url = "https://httpbin.org/json"
soup = fetch_html(url)

if soup:
    print("Título de la página:", soup.title.get_text(strip=True))


Título de la página https://books.toscrape.com/: All products | Books to Scrape - Sandbox
❌ Saltado: Content-Type no es HTML (application/json)


#### Buscar palabras clave / centinelas

🔹 Ejemplo práctico 1 (ver qué devuelve un sitio)

In [37]:
import requests

r = requests.get("https://httpbin.org/html")
print(r.text[:200])  # mostramos solo los primeros 200 caracteres


<!DOCTYPE html>
<html>
  <head>
  </head>
  <body>
      <h1>Herman Melville - Moby-Dick</h1>

      <div>
        <p>
          Availing himself of the mild, summer-cool weather that now reigned in t


In [38]:
if "<head>" in r.text.lower():
    print("Esta página tiene etiqueta <head>")
else:
    print("No tiene etiqueta <head>")


Esta página tiene etiqueta <head>


🌐 Contexto

Cuando scrapeas, a veces el servidor no devuelve lo que esperas.
En lugar de la página HTML normal, puedes recibir:

Un captcha (te pide resolver un puzzle para seguir).

Un mensaje de Access Denied / Forbidden / Blocked.

Una redirección a una página de login.

Aunque el Content-Type sea text/html, el contenido no sirve para tu análisis.

🕵️‍♂️ ¿Cómo detectarlo?

La idea es buscar palabras clave (centinelas) dentro del r.text.
Ejemplo clásico:

"access denied"

"forbidden"

"captcha"

"verify you are human"

"cloudflare" (a veces aparece cuando Cloudflare bloquea).

Estas frases funcionan como centinelas → si aparecen, sabemos que no es la página normal.

⚡ Ejemplo práctico

In [39]:
import requests

url = "https://httpbin.org/html"
r = requests.get(url)

# Pasamos todo a minúsculas para que no importe "Captcha" o "captcha"
texto = r.text.lower()

# Lista de palabras clave típicas de bloqueo
centinelas = ["access denied", "forbidden", "captcha", "verify you are human"]

# Verificamos si alguna aparece en el HTML
if any(c in texto for c in centinelas):
    print("❌ Página bloqueada o protegida")
else:
    print("✅ Página normal recibida")

# Para verlo mejor, mostramos los primeros 200 caracteres del HTML
print("\nHTML recibido (corto):\n", texto[:200])



✅ Página normal recibida

HTML recibido (corto):
 <!doctype html>
<html>
  <head>
  </head>
  <body>
      <h1>herman melville - moby-dick</h1>

      <div>
        <p>
          availing himself of the mild, summer-cool weather that now reigned in t


# 👉 Qué harás: forzar un encoding incorrecto y corregir con `apparent_encoding`.

![ss](encoding.png)

Ese "Ã±" es el típico síntoma de que alguien leyó UTF-8 como Latin-1.

In [41]:
import requests

# Simulamos un contenido en UTF-8 pero mal declarado
mal_contenido = "España, México, señorita".encode("utf-8")  

# El servidor "dice" que esto es Latin-1 (mal)
texto_mal = mal_contenido.decode("latin-1")  

print("❌ Texto roto (mal decodificado):", texto_mal)

# Si forzamos a usar utf-8 → lo reparamos
texto_bien = mal_contenido.decode("utf-8")  

print("✅ Texto correcto (decodificado bien):", texto_bien)


❌ Texto roto (mal decodificado): EspaÃ±a, MÃ©xico, seÃ±orita
✅ Texto correcto (decodificado bien): España, México, señorita


In [43]:

#¿Cómo funciona el or en Python?

#En Python, A or B devuelve el primer valor “verdadero” (truthy):
 
truthy = None or "ISO-8859-1"     # → "utf-8"
print(truthy)

truthy = "" or "utf-8"       # → "utf-8"
print(truthy)

ISO-8859-1
utf-8


Si r.apparent_encoding es, por ejemplo, "utf-8", se asigna eso.

Si r.apparent_encoding es None (o ""), se conserva r.encoding.

In [45]:
import requests
from bs4 import BeautifulSoup

r = requests.get("https://httpbin.org/encoding/utf8", timeout=10)

# 1️⃣ Detectar el encoding real
#r.encoding = r.apparent_encoding or r.encoding

if r.apparent_encoding:
    r.encoding = r.apparent_encoding
else:
    r.encoding = r.encoding  # (no cambia nada)


# 2️⃣ Parsear el HTML con BeautifulSoup
soup = BeautifulSoup(r.text, "html.parser")

# 3️⃣ Mostrar un trozo de texto plano
print("Primeros 580 chars:", soup.get_text(separator=" ")[:580])


Primeros 580 chars: Unicode Demo 
 Taken from  http://www.cl.cam.ac.uk/~mgk25/ucs/examples/UTF-8-demo.txt 
 

UTF-8 encoded sample plain-text file
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾

Markus Kuhn [ˈmaʳkʊs kuːn]   — 2002-07-25


The ASCII compatible UTF-8 encoding used in this plain-text file
is defined in Unicode, ISO 10646-1, and RFC 2279.


Using Unicode/UTF-8, you can write in emails and source code things such as

Mathematics and sciences:

  ∮ E⋅da = Q,  n → ∞, ∑ f(i) = ∏ g(i),      ⎧⎡⎛┌─────┐⎞⎤⎫
                                            ⎪⎢⎜│a²+b³ ⎟⎥⎪
  ∀x∈ℝ: ⌈x⌉ = −⌊−x⌋, α ∧ ¬β = ¬(¬α
